# App

In [1]:
from app import app

In [2]:
app.run_server(debug=True, port=8000)

# Data
*create data for pages*

In [4]:
import pandas as pd
import os

In [5]:
os.chdir('/home/lbk/works/pf')

In [6]:
from pf_utils import PortfolioManager as PM 

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [7]:
path_data = 'data'
path_tran = 'transaction'

In [8]:
file_cost='transaction_cost'

## Price

In [9]:
universe = 'UV_FUND'
fund = PM.create_universe(universe)
cost = PM.get_cost(universe, file=file_cost, path=path_tran)
fee = cost['fee']

64 securities from 2008-11-28 to 2025-01-31 uploaded.
REMINDER: 64 equities converted to daily
Daily metrics in Performance statistics must be meaningless
Price data loaded
Cost data transaction_cost_241217.csv loaded


In [10]:
df_prices = fund.df_prices
df_prices_fees = fund._get_prices_after_fee(df_prices, fee)

In [11]:
func = lambda x: [k for k,v in fund.security_names.items() if x.lower() in v.lower()]
cols = ['price', 'price_after_fees']
df_all = pd.DataFrame()
for i in range(2025,2060,5):
    tickers = func(str(i))
    df_a = pd.DataFrame()
    for j, df_p in enumerate([df_prices, df_prices_fees]):
        df = df_p[tickers].rename_axis('date')
        df = df.rename_axis('ticker',axis=1).unstack().to_frame(cols[j]).dropna()
        df_a = pd.concat([df_a, df], axis=1)
    df_a = df_a.assign(group=i).set_index('group', append=True).swaplevel(1,2).swaplevel(0,1)
    df_all = pd.concat([df_all, df_a])

df_all = (df_all.assign(ym=df_all.index.get_level_values("date").to_period('M'))
          .sort_index().groupby(['group','ticker','ym']).tail(1).drop('ym', axis=1))

In [17]:
date = df_all.index.get_level_values('date').max().strftime('%y%m%d')
date

'250131'

In [18]:
file = f'fund_monthly_{date}.csv'
path = 'pages'
df_all.to_csv(f'{path}/{file}')

In [19]:
file = f'fund_name_{date}.csv'
path = 'pages'
pd.Series(fund.security_names).to_csv(f'{path}/{file}')

In [20]:
file = f'fund_monthly_{date}.csv'
path = 'pages'
df_all = pd.read_csv(
    f'{path}/{file}',
    parse_dates=['date'],
    dtype={'ticker': str},
    index_col=['group', 'ticker', 'date']
)

In [21]:
file = f'fund_name_{date}.csv'
fund_name = pd.read_csv(f'{path}/{file}', dtype={'ticker': str}, index_col=[0]).to_dict()

## Bayesian

In [ ]:
import pandas as pd
import json
from pf_dash import get_inference

In [ ]:
file = 'fund_monthly_241229.csv'
path = 'pages'
df_prc = pd.read_csv(
    f'{path}/{file}',
    parse_dates=['date'],
    dtype={'ticker': str},
    index_col=['group', 'ticker', 'date']
)

In [ ]:
file = 'fund_name_241230.csv'
fund_name = pd.read_csv(f'{path}/{file}', dtype={'ticker': str}, index_col=[0]).squeeze().to_dict()

In [ ]:
kw_dst = dict(
    file = 'fund_f3y_s3y_250109.pkl',
    path = path_data,
    n_points=500, 
    error=0.9999
)
data = get_inference(**kw_dst)
df_dst = pd.DataFrame(data['density'], index=data['x'])
# confine to tickers in price data
df_dst = df_dst[list(fund_name.keys())]

In [ ]:
data_density = dict()
for group in df_prc.index.get_level_values('group').unique():
    tickers = df_prc.loc[group].index.get_level_values('ticker').unique().to_list()
    df = df_dst[tickers]
    df = df.loc[df.notna().any(axis=1)]
    df.columns = [fund_name[x] for x in df.columns] # ticker to name
    data_density[group] = {
        'density': df.to_dict('records'),
        'x': df.index.tolist(),
        'interval': {fund_name[k]:v for k,v in data['interval'].items() if k in tickers},
        'hdi_prob': data['hdi_prob'],
        'var_name': data['var_name']
    }

In [ ]:
file = 'fund_density_ret3y_250113.json'
path = 'pages'
with open(f'{path}/{file}', "w") as f:
    json.dump(data_density, f, indent=4)

In [ ]:
file = 'fund_density_ret3y_250113.json'
path = 'pages'
with open(f'{path}/{file}', "r") as f:
    data_density_json = f.read()  # Read raw JSON string directly

# check if successful
data_density = json.loads(data_density_json)
data_density = {int(k):v for k,v in data_density.items()}

*get back to pages*

In [ ]:
os.chdir('pages')

# Lab

## Data

In [ ]:
import pandas as pd

In [ ]:
file = 'fund_monthly_241229.csv'
path = '.'
df_prc = pd.read_csv(
    f'{path}/{file}',
    parse_dates=['date'],
    dtype={'ticker': str},
    index_col=['group', 'ticker', 'date']
)

In [ ]:
file = 'fund_name_241230.csv'
fund_name = pd.read_csv(f'{path}/{file}', dtype={'ticker': str}, index_col=[0]).iloc[:,0].to_dict()

## Bayesian

In [ ]:
from ddf_bayesian import BayesianEstimator

In [ ]:
df_p = df_prc['price_after_fees'].droplevel(0).unstack('ticker')

In [ ]:
freq_year = 3
days_in_year = 12 # months
freq = freq_year*days_in_year # months
date = '250109'

file_est = f'tdf_f{freq_year}y_{date}'
file_est

### Estimate

In [ ]:
kw = dict(
    days_in_year = days_in_year,
    security_names = fund_name
)
be = BayesianEstimator(df_p, **kw)

In [ ]:
be = BayesianEstimator.create(file_est, **kw)

## Analysis

In [ ]:
#xlims = None
xlims = [(-0.2, 0.6), (-1, 20)]
_ = be.plot_returns(max_legend=10, xlims=xlims)

In [ ]:
#var_name = 'cagr'
var_name = 'total_return'
ms = ['mean','sd','hdi_3%','hdi_97%']
ascending = [False, True, False, False]

df_s = be.bayesian_summary(var_name).droplevel(0)
pick = lambda i, n=5, df=df_s: df.sort_values(ms[i], ascending=ascending[i]).iloc[:n].index.to_list()

In [ ]:
#df_s.isna().any(axis=1).sum()
#df_s = df_s.dropna()

In [ ]:
t0 = pick(0)
kw = dict(
    ref_val='default', 
    figsize=(12,2.5), textsize=9, grid=(1,5), length=16)
be.plot_posterior(var_name, t0, **kw)

In [ ]:
t0

In [ ]:
tickers = t0

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde

n_points = 200

# Load posterior data
posterior = be.bayesian_data['trace'].posterior

# Specify variable and coordinates
param = "total_return"  # Example: multi-dimensional parameter
coords = {"ticker": tickers}  # Coordinates to filter

# Average over the chain dimension, keep the draw dimension
averaged_data = posterior[param].sel(**coords).mean(dim="chain")

# Convert to a DataFrame for Plotly
df = (averaged_data.stack(sample=["draw"])  # Combine draw dimension into a single index
      .to_pandas()  # Convert to pandas DataFrame
      .T)

# Example: KDE computation for the DataFrame
kde_data = []  # To store results
x_values = np.linspace(df.min().min(), df.max().max(), n_points)  # Define global x range

for ticker in df.columns:
    ticker_samples = df[ticker].values  # Extract samples for the ticker
    
    # Compute KDE
    kde = gaussian_kde(ticker_samples)
    density = kde(x_values)  # Compute density for the range
    
    # Store results in a DataFrame
    kde_data.append(pd.DataFrame({
        "x": x_values,
        "density": density,
        "Ticker": ticker
    }))

# Combine all KDE data into a single DataFrame
kde_df = pd.concat(kde_data, ignore_index=True)

# Plot using Plotly
fig = px.line(
    kde_df,
    x="x",
    y="density",
    color="Ticker",
    title=f"Density of {param.upper()}",
    #labels={"x": "Parameter Value", "density": "Density"},
)
fig.show()

In [ ]:
# Rename columns for clarity
df = df.rename(columns={param: "samples", "ticker": "Ticker"}).T

In [ ]:
import arviz as az
import pandas as pd
import plotly.express as px
import numpy as np

# Load posterior data
posterior = be.bayesian_data['trace'].posterior

# Specify variable and coordinates
param = "total_return"  # Example: multi-dimensional parameter
coords = {"ticker": tickers}  # Coordinates to filter

# Filter posterior samples based on coordinates
selected_data = posterior[param].sel(**coords)

# Stack chain and draw dimensions while keeping ticker
stacked_data = selected_data.stack(sample=("chain", "draw"))  # Combine chain and draw into "sample"

# Convert to a DataFrame for Plotly Express
df = (
    stacked_data.to_pandas()  # Convert to pandas DataFrame
    .reset_index()  # Reset index for easier handling
)

# Rename columns for better clarity
df = df.rename(columns={param: "samples", "ticker": "Ticker"})

In [ ]:
df = df.set_index('Ticker')
df

In [ ]:
# Average over the chain dimension, keep the draw dimension
averaged_data = posterior[param].sel(**coords).mean(dim="chain")
averaged_data

In [ ]:
# Convert to a DataFrame for Plotly
df = (
    averaged_data.stack(sample=["draw"])  # Combine draw dimension into a single index
    .to_pandas()  # Convert to pandas DataFrame
    #.reset_index()  # Flatten the DataFrame
)

In [ ]:
# Rename columns for clarity
df = df.rename(columns={param: "samples", "ticker": "Ticker"}).T

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde

# Example: KDE computation for the DataFrame
kde_data = []  # To store results
x_values = np.linspace(df.min().min(), df.max().max(), 500)  # Define global x range

for ticker in df.columns:
    ticker_samples = df[ticker].values  # Extract samples for the ticker
    
    # Compute KDE
    kde = gaussian_kde(ticker_samples)
    density = kde(x_values)  # Compute density for the range
    
    # Store results in a DataFrame
    kde_data.append(pd.DataFrame({
        "x": x_values,
        "density": density,
        "Ticker": ticker
    }))

# Combine all KDE data into a single DataFrame
kde_df = pd.concat(kde_data, ignore_index=True)


In [ ]:
# Plot using Plotly
fig = px.line(
    kde_df,
    x="x",
    y="density",
    color="Ticker",
    title=f"Density Plot of {param} (Averaged over Chains)",
    labels={"x": "Parameter Value", "density": "Density"},
)
fig.show()

In [ ]:
df.size

In [ ]:
kde_df.size

In [ ]:
px.histogram(df)

In [ ]:
df

In [ ]:
px.histogram(df)

In [ ]:
# Plot using Plotly Express
fig = px.histogram(
    df,
    x="samples",  # Values for the x-axis
    color="Ticker",  # Different colors for each ticker
    nbins=50,  # Number of bins for the histogram
    title=f"Posterior Distribution of {param} (Averaged over Chains)",
    marginal="rug",  # Add rug plot for individual samples
    labels={"samples": "Parameter Value"},
)

# Show plot
fig.show()

In [ ]:
# Plot using Plotly Express
fig = px.bar(
    df,
    x="Ticker",
    y="average_samples",
    title=f"Averaged Posterior Distribution of {param}",
    labels={"average_samples": "Average Value", "Ticker": "Ticker"},
)

# Show plot
fig.show()

In [ ]:
# Plot using Plotly Express
fig = px.histogram(
    df,
    x="samples",
    color="Ticker",  # Separate histograms by ticker
    nbins=50,
    title=f"Posterior Distribution of {param}",
    marginal="rug",  # Add rug plot for individual samples
    labels={"samples": "Parameter Value"},
)

# Add credible intervals for each coordinate (ticker)
for ticker in coords["ticker"]:
    ticker_samples = df[df["Ticker"] == ticker]["samples"]
    credible_interval = np.percentile(ticker_samples, [3, 97])  # 94% CI
    fig.add_vline(
        x=credible_interval[0],
        line_dash="dash",
        line_color="red",
        annotation_text=f"{ticker} 3%",
    )
    fig.add_vline(
        x=credible_interval[1],
        line_dash="dash",
        line_color="red",
        annotation_text=f"{ticker} 97%",
    )

# Show plot
fig.show()

In [ ]:
df.index.name = 'Ticker'
df

In [ ]:
df.columns.names

In [ ]:
df_flat = df.melt(
    id_vars=["chain", "Ticker"],  # Keep chain and Ticker as identifier variables
    var_name="draw",  # Name for the draw index
    value_name="samples"  # Name for the sample values
)
df_flat

In [ ]:
import arviz as az
import pandas as pd
import plotly.express as px
import numpy as np

# Load posterior data
posterior = be.bayesian_data['trace'].posterior

# Extract one parameter for demonstration
param = "mu"
samples = posterior[param].values.flatten()

# Create a DataFrame for Plotly Express
df = pd.DataFrame({"samples": samples})

# Plot using Plotly Express
fig = px.histogram(
    df,
    x="samples",
    nbins=50,
    title=f"Posterior Distribution of {param}",
    marginal="rug",  # Add rug plot for individual samples
    labels={"samples": "Parameter Value"},
)

# Add credible interval (e.g., 94%)
credible_interval = np.percentile(samples, [3, 97])
fig.add_vline(x=credible_interval[0], line_dash="dash", line_color="red", annotation_text="3%")
fig.add_vline(x=credible_interval[1], line_dash="dash", line_color="red", annotation_text="97%")

# Show plot
fig.show()


In [ ]:
df

# Deploy

In [26]:
if __name__ == '__main__':
    app.run_server(debug=True)

## Testing

In [14]:
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

# Initialize the Dash app
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)

# Tabs without any label_style initially
tabs_contents = [
    dbc.Tab(dcc.Graph(id='price-plot'), label='가격', id='tab-1'),
    dbc.Tab(dcc.Graph(id='return-plot'), label='수익률', id='tab-2'),
    dbc.Tab('tab_topic', label='토픽', id='tab-3', label_class_name="tab-label new-badge-label"),
    dbc.Tab('tab_notice', label='알림', id='tab-4'),
    dbc.Tab('tab_info', label='정보', id='tab-5'),
]
tabs = dbc.Tabs(tabs_contents, id='tabs')

# Layout with a hidden store component to hold tab IDs
app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(tabs),
        html.Br(),
        dcc.Location(id="url", refresh=False),  # To initialize the page
    ]
)

# Register the clientside callback to check if on mobile and apply label-style-mobile
app.clientside_callback(
    """
    function(pathname) {
        // Check if the current window width indicates a mobile device
        const isMobile = window.innerWidth < 768;
        const tabElement = document.getElementById('tab-2');
        
        if (tabElement) {
            // Add or remove the CSS class for label styling based on device type
            if (isMobile) {
                tabElement.classList.add('label-style-mobile');
            } else {
                tabElement.classList.remove('label-style-mobile');
            }
        }
        return window.dash_clientside.no_update;  // Return no update to children
    }
    """,
    Output('tabs', 'children'),  # Update the children of tabs (triggering the callback)
    Input('url', 'pathname'),
)

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)


In [16]:
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

# Initialize the Dash app
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)

# Tabs without any label_style initially
tabs_contents = [
    dbc.Tab(dcc.Graph(id='price-plot'), label='가격', id='tab-1'),
    dbc.Tab(dcc.Graph(id='return-plot'), label='수익률', id='tab-2'),
    dbc.Tab('tab_topic', label='토픽', id='tab-3', label_class_name="tab-label new-badge-label"),
    dbc.Tab('tab_notice', label='알림', id='tab-4'),
    dbc.Tab('tab_info', label='정보', id='tab-5'),
]
tabs = dbc.Tabs(tabs_contents, id='tabs')

# Layout with a hidden store component to hold tab IDs
app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(tabs),
        html.Br(),
        dcc.Location(id="url", refresh=False),  # To initialize the page
    ]
)

# Register the clientside callback to check if on mobile and apply label-style-mobile to all tabs
app.clientside_callback(
    """
    function(pathname) {
        // Check if the current window width indicates a mobile device
        const isMobile = window.innerWidth > 768;
        const tabElements = document.querySelectorAll('.nav-item');  // All tab items
        
        // Add or remove the CSS class for label styling for all tabs
        tabElements.forEach(function(tabElement) {
            if (isMobile) {
                tabElement.classList.add('label-style-mobile');
            } else {
                tabElement.classList.remove('label-style-mobile');
            }
        });
        
        return window.dash_clientside.no_update;  // Return no update to children
    }
    """,
    Output('tabs', 'children'),  # Update the children of tabs (triggering the callback)
    Input('url', 'pathname'),
)

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)


In [ ]:
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

# Initialize the Dash app
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)


# Tabs without any label_style initially
tabs_contents = [
    dbc.Tab(dcc.Graph(id='price-plot'), label='가격', id='tab-1'),
    dbc.Tab(dcc.Graph(id='return-plot'), label='수익률', id='tab-2', label_style='label-style-mobile'),
    dbc.Tab('tab_topic', label='토픽', id='tab-3', label_class_name="tab-label new-badge-label",
           label_style=label_style_mobile),
    dbc.Tab('tab_notice', label='알림', id='tab-4'),
    dbc.Tab('tab_info', label='정보', id='tab-5'),
]
tabs = dbc.Tabs(tabs_contents, id='tabs')

# Layout with a hidden store component to hold tab IDs
app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(tabs),
        html.Br(),
        dcc.Location(id="url", refresh=False),  # To initialize the page
        dcc.Store(id="tab-ids-store", data=[tab.id for tab in tabs_contents]),  # Store tab IDs
    ]
)

if __name__ == "__main__":
    app.run_server(debug=True)

In [1]:
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

# Initialize the Dash app
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)

label_style_mobile = {
    'padding-right': '0.5rem',
    'padding-left': '0.5rem'
}

# Tabs without any label_style initially
tabs_contents = [
    dbc.Tab(dcc.Graph(id='price-plot'), label='가격', id='tab-1'),
    dbc.Tab(dcc.Graph(id='return-plot'), label='수익률', id='tab-2', label_style=label_style_mobile),
    dbc.Tab('tab_topic', label='토픽', id='tab-3', label_class_name="tab-label new-badge-label"),
    dbc.Tab('tab_notice', label='알림', id='tab-4'),
    dbc.Tab('tab_info', label='정보', id='tab-5'),
]
tabs = dbc.Tabs(tabs_contents, id='tabs')

# Layout with a hidden store component to hold tab IDs
app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(tabs),
        html.Br(),
        dcc.Location(id="url", refresh=False),  # To initialize the page
    ]
)

if __name__ == "__main__":
    app.run_server(debug=True)

In [ ]:
app.clientside_callback(
    """
    function(tab_ids) {
        const viewportWidth = window.innerWidth;
        const isMobile = viewportWidth > 768;
    
        return tab_ids.map((existingClass, index) => {
            // Split the existing class string into an array
            let classList = existingClass ? existingClass.split(' ') : [];
            
            //console.log('Existing Class List:', classList);  // Debugging

            // Add 'tab-label' to all tabs
            if (!classList.includes('tab-label')) {
                classList.push('tab-label');
            }

            // Apply the 'new-badge-label' only to tab-3 (target based on index)
            if (index === 2) {  // tab-3 is at index 2 (0-based index)
                if (!classList.includes('new-badge-label')) {
                    classList.push('new-badge-label');  // Only add the badge to tab-3
                }
            }
            if (existingClass.includes('new-badge-label')) {
                if (!classList.includes('new-badge-label')) {
                    classList.push('new-badge-label');  // Add the badge class if already present
                }
            }

            // Dynamically add or remove 'mobile' class
            if (isMobile) {
                if (!classList.includes('mobile')) {
                    classList.push('mobile');
                }
            } else {
                classList = classList.filter(c => c !== 'mobile');
            }

            //console.log('Updated Class List:', classList);  // Debugging

            // Return the updated class list as a string
            return classList.join(' ').trim();
        });
    }
    """,
    [Output(tab.id, "label_style") for tab in tabs_contents],
    Input("tab-ids-store", "data"),
)

## Samples

### Callback

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, callback, Input, Output
from io import StringIO

file_prc = 'fund_monthly_241229.csv'
path = '.'

# Load price data
df_prc = pd.read_csv(
    f'{path}/{file_prc}',
    parse_dates=['date'],
    dtype={'ticker': str},
    index_col=['group', 'ticker', 'date']
)

app = Dash(__name__)
app.layout = html.Div([
    html.Button('Submit', id='submit-val', n_clicks=0),
    dcc.Graph(id='price-plot'),
    dcc.Store(id='price-data'),
])

group_value = 2030
col = 'price'
df_p = df_prc.loc[group_value, col].unstack('ticker').sort_index()

@callback(
    Output(component_id='price-data', component_property='data'),
    Input(component_id='submit-val', component_property='n_clicks')
)
def update(n_clicks):
    return df_p.to_json(date_format='iso', orient='split')


@callback(
    Output(component_id='price-plot', component_property='figure'),
    Input(component_id='price-data', component_property='data')
)
def plot(data):
    # Use StringIO to wrap the string data
    data_io = StringIO(data)
    df = pd.read_json(data_io, orient='split')
    return px.line(df)

if __name__ == '__main__':
    app.run_server(debug=True)

### Clientside

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc
import json

file_prc = 'fund_monthly_241229.csv'
path = '.'

# Load price data
df_prc = pd.read_csv(
    f'{path}/{file_prc}',
    parse_dates=['date'],
    dtype={'ticker': str},
    index_col=['group', 'ticker', 'date']
)

app = Dash(__name__)

group_value = 2030
col = 'price'
df_p = df_prc.loc[group_value, col].unstack('ticker').sort_index()

# Preprocess the data and serialize it to JSON
preprocessed_data = df_p.to_json(date_format='iso', orient='split')

app.layout = html.Div([
    dcc.Graph(id='price-plot'),
    # Embed the preprocessed data as a hidden div or dcc.Store
    dcc.Store(id='price-data', data=preprocessed_data),
])

# Clientside callback for plotting
app.clientside_callback(
    """
    function(data) {
        if (!data) {
            return {'data': [], 'layout': {}};
        }
        // Parse the JSON data
        const df = JSON.parse(data);
        const dates = df.index;
        const columns = df.columns;

        const traces = columns.map((col, i) => ({
            x: dates,
            y: df.data.map(row => row[i]),
            mode: 'lines',
            name: col
        }));

        return {
            data: traces,
            layout: {
                title: 'Price Plot',
                xaxis: {title: 'Date'},
                yaxis: {title: 'Price'}
            }
        };
    }
    """,
    Output('price-plot', 'figure'),
    Input('price-data', 'data')
)

if __name__ == '__main__':
    app.run_server(debug=True)


### Mobile

In [ ]:
import pandas as pd
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
import json

file_prc = 'fund_monthly_241229.csv'
path = '.'

# Load price data
df_prc = pd.read_csv(
    f'{path}/{file_prc}',
    parse_dates=['date'],
    dtype={'ticker': str},
    index_col=['group', 'ticker', 'date']
)

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

group_value = 2030
col = 'price'
df_p = df_prc.loc[group_value, col].unstack('ticker').sort_index()

# Preprocess the data and serialize it to JSON
preprocessed_data = df_p.to_json(date_format='iso', orient='split')

app.layout = dbc.Container(
    [
        dbc.Row(
            dbc.Col(
                html.Button('Submit', id='submit-val', n_clicks=0),
                width={"size": 6, "offset": 3},  # Centered on the screen
                className="text-center"
            )
        ),
        dbc.Row(
            dbc.Col(
                dcc.Graph(id='price-plot'),
                width=12  # Full-width on all screen sizes
            ),
            className="mt-4"  # Add margin-top for spacing
        ),
        # Store the preprocessed data
        dcc.Store(id='price-data', data=preprocessed_data),
    ],
    fluid=True  # Full-width container
)

# Clientside callback for plotting
app.clientside_callback(
    """
    function(data) {
        if (!data) {
            return {'data': [], 'layout': {}};
        }
        // Parse the JSON data
        const df = JSON.parse(data);
        const dates = df.index;
        const columns = df.columns;

        const traces = columns.map((col, i) => ({
            x: dates,
            y: df.data.map(row => row[i]),
            mode: 'lines',
            name: col
        }));

        return {
            data: traces,
            layout: {
                title: 'Price Plot',
                xaxis: {title: 'Date'},
                yaxis: {title: 'Price'},
                margin: {l: 40, r: 10, t: 40, b: 40},
                responsive: true
            }
        };
    }
    """,
    Output('price-plot', 'figure'),
    Input('price-data', 'data')
)

if __name__ == '__main__':
    app.run_server(debug=True)


## testing

## testing

## testing

In [ ]:
import pandas as pd

In [ ]:
file = 'fund_name_241230.csv'
path = '.'

fund_name = pd.read_csv(f'{path}/{file}', dtype={'ticker': str}, index_col=[0])
fund_name = fund_name.iloc[:,0].to_dict()
fund_name

In [ ]:
import re


m = re.search(r'\d{4}', s)
func = lambda s: [s if x is None else s[:x.end()] for x in [re.search(r'\d{4}', s)]][0]

In [ ]:
s = '신한마음편한TDF2040증권투자신탁[주식혼합-재간접형](종류C-re)'
#s = '신한마음편한TDF증권투자신탁[주식혼합-재간접형](종류C-re)'

func = lambda s: s[:m.end()] if (m := re.search(r'\d{4}', s)) else s

func(s)

In [ ]:
#i = 2050
i = 2055
data = preprocessed_data[i]
#data.keys()

col = '수수료 적용 전'
d = data['default'][col]['history']
i = data['default'][col]['index']
df = pd.DataFrame(d, index=i)

import plotly.express as px
px.line(df)

In [ ]:
import plotly.graph_objects as go

i = 2050
data = preprocessed_data[i]
#data.keys()

col = '수수료 적용 전'
d = data['default'][col]['history']
i = data['default'][col]['index']

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=i, y=d,
                    mode='lines',
                    #name='ticker'
                        )
             )
fig.show()

In [ ]:
# You can use Plotly's `plot` function to visualize the result
import plotly.offline as pyo
pyo.plot(result)

In [ ]:
d

In [ ]:
"""
            return {
                x: dat.index,        // Dates from the index
                y: yValues,          // Price history for each ticker
                type: 'scatter',
                mode: 'lines',
                name: ticker,        // Ticker as the legend name
                //color: ticker
            };
            """

In [ ]:
import plotly.graph_objects as go

# Create random data with numpy
import numpy as np
np.random.seed(1)

N = 100
random_x = np.linspace(0, 1, N)
random_y0 = np.random.randn(N) + 5
random_y1 = np.random.randn(N)
random_y2 = np.random.randn(N) - 5

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=random_x, y=random_y0,
                    mode='markers',
                    name='markers'))


"""
            return {
                x: dat.index,        // Dates from the index
                y: yValues,          // Price history for each ticker
                type: 'scatter',
                mode: 'lines',
                name: ticker,        // Ticker as the legend name
                //color: ticker
            };
            """

In [ ]:
import plotly.graph_objects as go
import pandas as pd

data= pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/2014_usa_states.csv")

fig = go.Figure(data=go.Scatter(x=data['Postal'],
                                y=data['Population'],
                                mode='markers',
                                marker_color=data['Population'],
                                text=data['State'])) # hover text goes here

fig.update_layout(title=dict(text='Population of USA States'))
fig.show()

In [ ]:
import plotly.express as px

df = px.data.gapminder().query("country=='Canada'")
fig = px.line(df, x="year", y="lifeExp", title='Life expectancy in Canada', color='country')
fig.show()

In [ ]:
df = px.data.gapminder().query("continent=='Oceania'")
fig = px.line(df, x="year", y="lifeExp", color='country')
fig.show()